Пусть задано поле $M \subset \mathbb{B}^{n \times n}$, где $\mathbb{B} = \{ \mathbf{0}, \mathbf{1} \}$ --- логический носитель, а  $n = 7$ --- размер поля. (Само множество $M$
является булевой матрицей размеров $n \times n$). На поле $M$ произвольным (но разумным) образом можно нарисовать символ "крестика" или "нолика"; само поле, таким образом, представляет собой одну клетку рабочего пространства игры Крестики-Нолики.

Необходимо реализовать на языке программирования Python программный компонент, моделирующий единственный нейрон с $n^2$ входами и одним выходом. Результат работы модели $a(M) \subset \{  \textbf{O-class}, \textbf{X-class} \}$ --- распознанный класс "крестик" или "нолик".

Разработка модели $a(M)$ предполагает:
1. **Формирование обучающей выборки**. Размер обучающей выборки, характеристики прецедентов подбирается самостоятельно исходя из принципа рациональности и имея ввиду то обстоятельство, то как крестики, так и нолики могут быть нарисованы на поле вручную.
2. **Обучение модели методом стохастического градиента**. Требуется самостоятельно подобрать параметры обучения (в том числе и функцию активации $\varphi(x)$), а также разбиения обучающей выборки на тренировочную часть и тестовую. Обученная модель должна демонстрировать хорошие показатели качества: *точность и полнота не менее 98.5%*.
3. Модель должна предусматривать стандартные функции для работы с моделями --- обучение (fit), применение/прогноз (predict) и оценки качества (score) в части точности и полноты.
4. Модель должна предусматривать функцию сериализации (serialization) и десериализации (deserialization) в файл в формате JSON. Иными словами, необходимо обеспечить функциональность сохранения обученной модели в файл, а также его последующей загрузки.
	
В программных компонентах должно быть предусмотрено проведение всех необходимых математических операций и преобразований. Исключается использование готовых компонент и/или библиотек, позволяющих решить поставленную задачу "из коробки". Допускается использовать библиотеку numpy и pandas.

In [127]:
# импорт основных классов
import numpy as np
import random as random
import logging as log
import math as ma
import json
import codecs

from termcolor import colored, cprint
import random

In [65]:
#создаем класс, который генерит крестики и нолики
class createXO(object):
    
    def __init__(self):
        self.matrix = [[0]*7 for i in range(7)]
        pass
    
    def createX(self):
      # задаем начальную точку для рисования крестика
        x = random.randint(2,4)
        y = random.randint(2,4)
        self.matrix[x][y] = 1
        # задаем радиус рисования крестика
        rad = min(6-x,6-y, x, y)
        # заполняем крестик по диагоналям
        for i in range(1, rad):
            self.matrix[x-i][y-i] = 1
            self.matrix[x-i][y+i] = 1
            self.matrix[x+i][y-i] = 1
            self.matrix[x+i][y+i] = 1
        # оставшиеся клетки заполняем случайно 0/1
        self.matrix[x+rad][y-rad] = random.randint(0,1)
        self.matrix[x-rad][y+rad] = random.randint(0,1)
        self.matrix[x+rad][y+rad] = random.randint(0,1)
        self.matrix[x-rad][y-rad] = random.randint(0,1)
        return self.matrix
    
    def createO(self):
        x = random.randint(2,4)
        y = random.randint(2,4)
        rad = min(6-x,6-y, x, y)
        for i in range(-rad+1, rad):
            self.matrix[x+i][y-rad] = 1
            self.matrix[x+i][y+rad] = 1
            self.matrix[x+rad][y+i] = 1
            self.matrix[x-rad][y+i] = 1
        self.matrix[x+rad][y-rad] = random.randint(0,1)
        self.matrix[x-rad][y+rad] = random.randint(0,1)
        self.matrix[x+rad][y+rad] = random.randint(0,1)
        self.matrix[x-rad][y-rad] = random.randint(0,1)
        return self.matrix

In [97]:
# размер данных
data_value = 1000

# размер элемента данных
matrix_len=7

# заполняем нулями матрицу для данных
data = np.zeros(((data_value),(matrix_len*matrix_len+1)))
labels = np.zeros(data_value)
number = 0

# заполняем матрицу данных элементами 0/1
for h in range(data_value):
    ac = createXO()

    # выбираем элемнт случайным образом
    choose = random.randint(0,1)
    labels[h] = choose
    if choose == 0:
        m=ac.createO()

    if choose == 1:
        m=ac.createX()

    # в матрицу данных 0/1 кладем по строкам -> одной строкой один элемент
    for l in range(matrix_len):
        for p in range(matrix_len):
            data[h][p + matrix_len*l] = m[l][p]

    data[h][matrix_len*matrix_len] = -1.0

In [67]:
def printXO(data):
    for j in range(0, matrix_len):
        for x in data[j * matrix_len: (j + 1) * matrix_len]:
            print(colored(str(int(x)), 'red'), end=' ') if x == 1 else print(int(x), end=' ')
        print()

# i = random.randint(0, len(data))
# printXO(data[i])

In [68]:
# предсказываем результат определения элемента, на вход элемент и используемые веса
def predict(row, weights):
    activation = dot_product(row, weights)
    return 1 if activation >= 0.0 else 0

In [87]:
def loss(y1, y2):
    return (y1-y2) ** 2

def loss_deriv(y1, y2):
    return 2 * (y1-y2)

def quality(data,lab,weights):
    qual = 0.0
    data_size = len(lab)
    for h in range(data_size):
      qual += loss(a(data[h],weights),lab[h])

    return qual

In [88]:
def phi(z):
  return 1.0 / (1.0 + ma.exp(-z))

def phi_deriv(z):
  return phi(z) * (1.0 - phi(z))

def dot_product(precedent, weights):
  field = 0.0
  for l in range(matrix_len):
    for p in range(matrix_len):
      field += precedent[p + matrix_len*l]*weights[p + matrix_len*l]
    
  field += precedent[matrix_len*matrix_len]*weights[matrix_len*matrix_len] # w0
  return field;

def a(precedent, weights):
  return phi(dot_product(precedent, weights))

def a_phi_deriv(precedent, weights):
  return phi_deriv(dot_product(precedent, weights))

In [113]:
train_log = []
# обучение нейрона
def train_weights(train, train_labels, test, test_labels, l_rate, n_epoch):
    
    # иициализируем веса в начале нулями
    dim = len(train[0])
    weights = [((random.random()*1.0/dim)-1.0/(2.0*dim)) for i in range(dim)]
    data_size = len(train_labels)
    qual = quality(train,train_labels,weights)

    for epoch in range(0,n_epoch):
      qual = quality(train,train_labels,weights)
      for i in range(data_size):
        x_i = train[i]
        a_i = a(x_i,weights)
        y_i = train_labels[i]
        e_i = loss(a_i,y_i)

        step = loss_deriv(a_i,y_i)*a_phi_deriv(x_i,weights)*l_rate * x_i # vector
        weights -= step

        qual = ((data_size-1) / data_size) * qual + (1.0 / data_size) * (e_i ** 2)
      print("Epoch " + str(epoch) + ":")
      print("Train loss = " + str(qual))
      print("Test acc = " +str(calculate_accuracy(test,test_labels,weights)))   
            
    return weights

In [112]:
def calculate_accuracy(data,labels,weights):
  size = len(labels)
  right = 0
  for i in range(size):
    x_i = data[i]
    y_i = labels[i]
    p = predict(x_i,weights)
    if p == labels[i]:
      right += 1
  
  accuracy = right/size
  return accuracy

In [205]:
def serialize_model(weights):
  b = weights.tolist() # nested lists with same data, indices
  file_path = "model.json" ## your path variable
  with open(file_path, "w") as text_file:
    text_file.write(json.dumps(b))

In [209]:
def deserialize_model():
  file_path = "model.json" ## your path variable
  obj_text = ""
  with open(file_path, "r") as text_file:
    obj_text = text_file.read()
  b_new = json.loads(obj_text)
  a_new = np.array(b_new)
  return a_new;

In [247]:
spl = 0.7
sample = int(spl*data_value)

train_idx = np.random.choice(data_value, sample)
Xtrain = data[train_idx]
Ytrain = labels[train_idx]

test_idx = [idx for idx in range(data_value) if idx not in train_idx]
Xtest = data[test_idx]
Ytest = labels[test_idx]

weights_f = train_weights(Xtrain,Ytrain,Xtest,Ytest,0.01,10)
weights_f

Epoch 0:
Train loss = 64.98729728965183
Test acc = 0.7579365079365079
Epoch 1:
Train loss = 33.91996035672062
Test acc = 0.7599206349206349
Epoch 2:
Train loss = 22.145244157882857
Test acc = 0.753968253968254
Epoch 3:
Train loss = 15.933785315573427
Test acc = 0.753968253968254
Epoch 4:
Train loss = 12.228966664560904
Test acc = 0.753968253968254
Epoch 5:
Train loss = 9.82052889237097
Test acc = 0.753968253968254
Epoch 6:
Train loss = 8.15167018017742
Test acc = 0.746031746031746
Epoch 7:
Train loss = 6.937609129705391
Test acc = 0.746031746031746
Epoch 8:
Train loss = 6.020157090827253
Test acc = 0.746031746031746
Epoch 9:
Train loss = 5.305453400507753
Test acc = 0.746031746031746


array([ 0.33788885, -0.2193745 , -0.459321  , -0.78669768, -0.52346627,
       -0.1419871 ,  0.18699853, -0.05109731, -0.10660369, -0.125693  ,
       -0.07351031, -0.07322126, -0.09445958, -0.25188527, -0.47148177,
       -0.0902259 , -0.38163957,  0.4358913 , -0.2422992 , -0.12057157,
       -0.42826565, -0.77858344, -0.07646066,  0.48841002,  1.74526724,
        0.39285454, -0.03121706, -0.77930945, -0.55140261, -0.2018665 ,
       -0.36428888,  0.44738985, -0.35781998, -0.13290955, -0.41020966,
       -0.19563236,  0.01136338, -0.10083902,  0.04416315, -0.14930473,
       -0.03606433, -0.2309536 ,  0.20794573, -0.17915055, -0.47533964,
       -0.80110981, -0.40899292, -0.22412707,  0.27210541, -1.05096959])

In [243]:
serialize_model(weights_f)

[0.2866024469310081, -0.16618845697089143, -0.4236634160439451, -0.7672829569040851, -0.4297652016507441, -0.24786593721300626, 0.22654860679125988, -0.13334710659010043, -0.02493823585428538, -0.10455018727370974, -0.056360263907993174, -0.156350150240717, -0.016524702457475234, -0.23245084827854234, -0.5076695665272998, -0.1245450735696021, -0.3151778282785259, 0.41711772186082074, -0.24771956441951717, -0.14046946198612675, -0.4135849518418876, -0.7869090762996309, -0.012005151659699575, 0.44787096264194876, 1.704839479891663, 0.47357976435569, -0.09376022851592458, -0.8090461675572899, -0.558843183504183, -0.1558218483622171, -0.3264661051630351, 0.482536309384983, -0.3736290641655969, -0.10577059446206356, -0.46574516475824024, -0.1638545239652068, -0.015182109232116362, -0.11968670611223112, -0.09339832183681744, -0.0964523853802545, -0.054320027035623906, -0.182937872686778, 0.2288445348687887, -0.3079717299570366, -0.5039265796980581, -0.8018402285230685, -0.42040709238942287, 

In [251]:
!cat model.json

[0.2866024469310081, -0.16618845697089143, -0.4236634160439451, -0.7672829569040851, -0.4297652016507441, -0.24786593721300626, 0.22654860679125988, -0.13334710659010043, -0.02493823585428538, -0.10455018727370974, -0.056360263907993174, -0.156350150240717, -0.016524702457475234, -0.23245084827854234, -0.5076695665272998, -0.1245450735696021, -0.3151778282785259, 0.41711772186082074, -0.24771956441951717, -0.14046946198612675, -0.4135849518418876, -0.7869090762996309, -0.012005151659699575, 0.44787096264194876, 1.704839479891663, 0.47357976435569, -0.09376022851592458, -0.8090461675572899, -0.558843183504183, -0.1558218483622171, -0.3264661051630351, 0.482536309384983, -0.3736290641655969, -0.10577059446206356, -0.46574516475824024, -0.1638545239652068, -0.015182109232116362, -0.11968670611223112, -0.09339832183681744, -0.0964523853802545, -0.054320027035623906, -0.182937872686778, 0.2288445348687887, -0.3079717299570366, -0.5039265796980581, -0.8018402285230685, -0.42040709238942287, 

In [248]:
weights_f = deserialize_model()

In [250]:
calculate_accuracy(Xtest,Ytest,weights_f)

0.7599206349206349